In [ ]:
!pip install app-store-scraper
!pip install itunes-app-scraper-dmi
!pip install pymongo[srv]

In [ ]:
import pandas as pd

# for scraping app info from App Store
from itunes_app_scraper.scraper import AppStoreScraper

# for scraping app reviews from App Store
from app_store_scraper import AppStore

# for pretty printing data structures
from pprint import pprint

# for keeping track of timing
import datetime as dt
from tzlocal import get_localzone

# for building in wait times
import random
import time

## Set up loop to go through all apps
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import datetime                            # Imports datetime library

import pymongo
from pymongo import MongoClient

user = 'alfalimbany'
sandi = 'BangRaf454647'
# uri (uniform resource identifier) defines the connection parameters

uri = 'mongodb+srv://'+ user +':'+ sandi +'@kuliah.uohhxud.mongodb.net/?retryWrites=true&w=majority'
# start client to connect to MongoDB server
client = MongoClient(uri)

In [ ]:
client.stats                                # .stats  show details about the client

Database(MongoClient(host=['ac-k0260yr-shard-00-00.uohhxud.mongodb.net:27017', 'ac-k0260yr-shard-00-02.uohhxud.mongodb.net:27017', 'ac-k0260yr-shard-00-01.uohhxud.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-or6xar-shard-0', tls=True), 'stats')

In [ ]:
App_store_db = client['App_store_db']

info_collection = App_store_db['info_collection']

comment_collection = App_store_db['comment_collection']

In [ ]:
## Read in file containing app names and IDs
app_df = pd.read_csv('/content/drive/MyDrive/Kuliah/RPL/scraper/AppStore/apps.csv')
app_df.head()

,Unnamed: 0,app_name,iOS_revs,iOS_app_name,iOS_app_id
0,0,Spongebob Squarepants,124,spongebob-squarepants,1523633394
1,1,Spongebob Idle Adventures,1117,spongebobs-idle-adventures,1533163131


In [ ]:
## Get list of app names and app IDs
app_names = list(app_df['iOS_app_name'])
app_ids = list(app_df['iOS_app_id'])

In [ ]:
## Set up App Store Scraper
scraper = AppStoreScraper()
app_store_list = list(scraper.get_multiple_app_details(app_ids))

## Pretty print the data for the first app
pprint(app_store_list[0])

{'advisories': 'Soms/Mild grof taalgebruik of grove humor,Soms/Mild '
               'animatiegeweld of fictief geweld',
 'appletvScreenshotUrls': '',
 'artistId': 316574911,
 'artistName': 'HandyGames',
 'artistViewUrl': 'https://apps.apple.com/nl/developer/handygames/id316574911?uo=4',
 'artworkUrl100': 'https://is1-ssl.mzstatic.com/image/thumb/Purple116/v4/24/be/b2/24beb282-3c76-77fa-9b13-0b9788794b3e/AppIcon-0-0-1x_U007emarketing-0-0-0-7-0-0-sRGB-0-0-0-GLES2_U002c0-512MB-85-220-0-0.png/100x100bb.jpg',
 'artworkUrl512': 'https://is1-ssl.mzstatic.com/image/thumb/Purple116/v4/24/be/b2/24beb282-3c76-77fa-9b13-0b9788794b3e/AppIcon-0-0-1x_U007emarketing-0-0-0-7-0-0-sRGB-0-0-0-GLES2_U002c0-512MB-85-220-0-0.png/512x512bb.jpg',
 'artworkUrl60': 'https://is1-ssl.mzstatic.com/image/thumb/Purple116/v4/24/be/b2/24beb282-3c76-77fa-9b13-0b9788794b3e/AppIcon-0-0-1x_U007emarketing-0-0-0-7-0-0-sRGB-0-0-0-GLES2_U002c0-512MB-85-220-0-0.png/60x60bb.jpg',
 'averageUserRating': 4.40909,
 'averageUserRati

In [ ]:
## Convert list of dicts to Pandas DataFrame and write to csv
app_info_df = pd.DataFrame(app_store_list)
info_collection.insert_many(app_store_list) #MongoDB
app_info_df.to_csv('/content/drive/MyDrive/Kuliah/RPL/scraper/AppStore/appsDetail.csv', index=False) #CSV
app_info_df.head()

,isGameCenterEnabled,supportedDevices,features,advisories,screenshotUrls,ipadScreenshotUrls,appletvScreenshotUrls,artworkUrl512,artworkUrl60,artworkUrl100,...,userRatingCountForCurrentVersion,trackContentRating,minimumOsVersion,contentAdvisoryRating,averageUserRating,trackCensoredName,trackViewUrl,version,wrapperType,userRatingCount
0,True,"iPhone5s-iPhone5s,iPadAir-iPadAir,iPadAirCellu...","gameCenter,iosUniversal","Soms/Mild grof taalgebruik of grove humor,Soms...",https://is1-ssl.mzstatic.com/image/thumb/Purpl...,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,...,22,9+,13.0,9+,4.40909,SpongeBob SquarePants,https://apps.apple.com/nl/app/spongebob-square...,1.2.4,software,22
1,False,"iPhone5s-iPhone5s,iPadAir-iPadAir,iPadAirCellu...",iosUniversal,,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,...,301,4+,13.0,4+,4.56811,SpongeBob’s Idle Adventures,https://apps.apple.com/nl/app/spongebobs-idle-...,1.117,software,301


In [ ]:
for app_name, app_id in zip(app_names, app_ids):

    # Get start time
    start = dt.datetime.now(tz=get_localzone())
    fmt= "%m/%d/%y - %T %p"

    # Print starting output for app
    print('---'*20)
    print('---'*20)
    print(f'***** {app_name} started at {start.strftime(fmt)}')
    print()

    # Instantiate AppStore for app
    app_ = AppStore(country='id', app_name=app_name, app_id=app_id)

    # Scrape reviews posted since February 28, 2020 and limit to 10,000 reviews
    app_.review(how_many=10000,
                after=dt.datetime(2020, 2, 28),
                sleep=random.randint(20,25))

    reviews = app_.reviews

    # Add keys to store information about which app each review is for
    for rvw in reviews:
        rvw['app_name'] = app_name
        rvw['app_id'] = app_id

    # Print update that scraping was completed
    print(f"""Done scraping {app_name}.
    Scraped a total of {app_.reviews_count} reviews.\n""")

    # Convert list of dicts to Pandas DataFrame and write to csv
    output_path = '/content/drive/MyDrive/Kuliah/RPL/scraper/AppStore/' + app_name + '.csv'
    #output_path = 'Var/AppStore/' + app_name + '.csv'
    review_df = pd.DataFrame(reviews)
    review_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path))

    #save into MongoDB
    comment_collection.insert_many(reviews)

    # Get end time
    end = dt.datetime.now(tz=get_localzone())

    # Print ending output for app
    print(f"""Successfully wrote {app_name} reviews to csv
    at {end.strftime(fmt)}.\n""")
    print(f'Time elapsed for {app_name}: {end-start}')
    print('---'*20)
    print('---'*20)
    print('\n')

    # Wait 5 to 10 seconds to start scraping next app
    time.sleep(random.randint(5,10))

------------------------------------------------------------
------------------------------------------------------------
***** spongebob-squarepants started at 12/29/23 - 02:52:31 AM

Done scraping spongebob-squarepants. 
    Scraped a total of 33 reviews.

Successfully wrote spongebob-squarepants reviews to csv
    at 12/29/23 - 02:52:55 AM.

Time elapsed for spongebob-squarepants: 0:00:23.181453
------------------------------------------------------------
------------------------------------------------------------


------------------------------------------------------------
------------------------------------------------------------
***** spongebobs-idle-adventures started at 12/29/23 - 02:53:00 AM

Done scraping spongebobs-idle-adventures. 
    Scraped a total of 6 reviews.

Successfully wrote spongebobs-idle-adventures reviews to csv
    at 12/29/23 - 02:53:01 AM.

Time elapsed for spongebobs-idle-adventures: 0:00:01.518364
-----------------------------------------------------